In [1]:
#!pip install spacy-langdetect
#!pip install language-detector
#!pip install symspellpy
#!pip install sentence-transformers

# 1. Read dataset.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
meta = pd.read_csv("../metadata.csv")
print(meta.shape)
meta.head()

C:\Users\zhipe\Anaconda3\envs\tm\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(341713, 19)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


Filter meta file by selecting only papers after 2020.

In [4]:
meta["publish_time"] = pd.to_datetime(meta["publish_time"])
meta["publish_year"] = pd.DatetimeIndex(meta['publish_time']).year
meta["publish_month"] = pd.DatetimeIndex(meta['publish_time']).month
meta = meta[meta["publish_year"] == 2020]
print(meta.shape[0], " papers are available after 2020 Jan 1.")

247255  papers are available after 2020 Jan 1.


Create dataframe with all abstracts and use it as input corpus.

In [5]:
index = []
for i in range(len(meta)):
    if type(meta.iloc[i, 8]) != float:
        index.append(i)
docs = meta.iloc[index, 8]
docs = docs.reset_index(drop= True)
print(docs.shape)
docs.head()

(163820,)


0    BACKGROUND: Dexmedetomidine has been reported ...
1    BACKGROUND: Global end-diastolic volume (GEDV)...
2    BACKGROUND: Human metapneumovirus (HMPV) is an...
3    BACKGROUND: Antimicrobial resistance (AMR) com...
4    Glycoconjugate vaccines based on bacterial cap...
Name: abstract, dtype: object

In [6]:
samp_size = 5100 #51000
samp_docs = docs.sample(n=samp_size,random_state=100)

# 2. Preprocessing texts

Sentence level preprocess
* Lowercase + base filter
* Some basic normalization

In [7]:
import re
def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    # Add missing delimiter: xxxThis is a --> xxx.
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', rw)  # before lower case
    # Make words all lower case.
    s = s.lower()
    # Remove < and >
    s = re.sub(r'&gt|&lt', ' ', s)
    # Remove letters that repeat more than 2 time.
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # Remove non-word characters that repeat more than 1 time.
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # Use string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # Remove words in parenthesis, which are assumed less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # [.?!]xxx --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    return s.strip()

text_lists = [preprocess_sent(text) for text in samp_docs]

 Word level preprocess

In [8]:
# define stemmer
import nltk
from nltk.stem.porter import *
p_stemmer = PorterStemmer()

# create stop words list
from stop_words import get_stop_words
stop_words = (list(
    set(get_stop_words('en'))
    |set(get_stop_words('es'))
    |set(get_stop_words('de'))
    |set(get_stop_words('it'))
    |set(get_stop_words('ca'))
    |set(get_stop_words('pt'))
    |set(get_stop_words('pl'))
    |set(get_stop_words('da'))
    |set(get_stop_words('ru'))
    |set(get_stop_words('sv'))
    |set(get_stop_words('sk'))
    |set(get_stop_words('nl'))
    |set(["abstract"])
))

In [9]:
from nltk.tokenize import word_tokenize
def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = [word for word in w_list if word.isalpha()]
    w_list = [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']
    w_list = [word for word in w_list if word not in stop_words]
    w_list = [p_stemmer.stem(word) for word in w_list]

    return w_list

token_lists = [preprocess_word(text) for text in text_lists]
print(len(token_lists))

In [ ]:
token_lists = [item for item in token_lists if item != []]
print(len(token_lists))

In [16]:
# samp_pd = pd.DataFrame({"abstract": samp_docs, "token":token_lists}).reset_index(drop= True)
# print(samp_pd.shape)
# samp_pd.replace([],np.nan,inplace=True)
# samp_pd.dropna(subset=["token"],inplace=True)
# print(samp_pd.shape)
# samp_pd = samp_pd[samp_pd.token]
# samp_pd.head()

(5100, 2)
(5100, 2)


# 4. Define model object

In [23]:
import gensim
from gensim import corpora, models

In [24]:
ntopic = 10
dictionary = corpora.Dictionary(token_lists)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in token_lists]
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=ntopic, random_state=66, id2word=dictionary, passes=20)

In [25]:
for topic in lda_model.print_topics():
    print(topic)

(0, '0.037*"protein" + 0.018*"interact" + 0.014*"structur" + 0.013*"mutat" + 0.013*"compound" + 0.011*"domain" + 0.010*"spike" + 0.010*"bind" + 0.010*"host" + 0.010*"studi"')
(1, '0.034*"test" + 0.030*"sampl" + 0.025*"student" + 0.022*"detect" + 0.020*"method" + 0.017*"assay" + 0.017*"sensit" + 0.016*"antibodi" + 0.015*"result" + 0.014*"school"')
(2, '0.030*"health" + 0.023*"care" + 0.015*"pandem" + 0.012*"system" + 0.011*"servic" + 0.010*"practic" + 0.009*"commun" + 0.008*"manag" + 0.007*"respons" + 0.007*"healthcar"')
(3, '0.048*"cell" + 0.021*"infect" + 0.019*"respons" + 0.014*"express" + 0.014*"diseas" + 0.012*"gene" + 0.012*"immun" + 0.011*"cytokin" + 0.011*"lung" + 0.011*"role"')
(4, '0.102*"patient" + 0.021*"diseas" + 0.016*"case" + 0.015*"risk" + 0.015*"studi" + 0.015*"treatment" + 0.014*"group" + 0.014*"result" + 0.014*"infect" + 0.013*"hospit"')
(5, '0.040*"case" + 0.029*"model" + 0.020*"number" + 0.020*"data" + 0.018*"countri" + 0.018*"transmiss" + 0.017*"rate" + 0.016*"infe

the Dominant topic and its percentage contribution in each document

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)